In [13]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

import pickle
import pandas as pd
import time

from utils import greedy, simplex_projection, integer_initial_vector
from continuous import solve_continuous_quadratic

In [14]:
from utils import projection

n = 100
setting = f'n{n}'

for obj in ['f', 'crash', 'fuel']:
    results = []

    for alpha in [0.1, 0.2, 0.5, 1, 2, 5, 10]:
        params = np.load(f'data/{obj}-{setting}/o{alpha}/params.npz')
        R = params['R']

        for trial in range(10):
            yinit = integer_initial_vector(n, R)
            y = integer_initial_vector(n, R)
            pred = integer_initial_vector(n, R)

            for t in range(100):
                with open(f'data/{obj}-{setting}/o{alpha}/{trial*100+t}.pkl', 'rb') as f:
                    G = pickle.load(f)

                # Cold
                start = time.time()
                itr_c, _ = greedy(yinit, G)
                time_c = time.time() - start

                # Learn
                start = time.time()
                itr_p, x = greedy(pred, G)
                time_p = time.time() - start

                # online subgradient descent update
                y += .01 * (R/np.sqrt(n)) * np.sign(x - y)
                y = simplex_projection(y, R)
                pred = (pred*(t+1) + y)/(t+2)

                # Relax
                qsol, time_qc = solve_continuous_quadratic(G, yinit)
                start = time.time()
                itr_q, _ = greedy(qsol, G)
                time_qg = time.time() - start
                time_q = time_qc + time_qg

                results.append([alpha, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg])

                print(obj, alpha, trial, t, itr_c, itr_p, itr_q, time_c, time_p, time_q, time_qc, time_qg)

    # create a DataFrame from the results list
    df = pd.DataFrame(results, columns=['alpha', 'trial', 't', 'itr_c', 'itr_p', 'itr_q', 'time_c', 'time_p', 'time_q', 'time_qc', 'time_qg'])
    #save df as pickle in result folder
    df.to_pickle(f'result/{obj}-{setting}.pkl')

f 0.1 0 0 192 192 7 0.13796520233154297 0.13477873802185059 0.01254725456237793 0.005448102951049805 0.007099151611328125
f 0.1 0 1 192 157 7 0.13742899894714355 0.11245584487915039 0.01275491714477539 0.00514984130859375 0.007605075836181641
f 0.1 0 2 192 115 7 0.13898205757141113 0.08316516876220703 0.012337446212768555 0.005236148834228516 0.007101297378540039
f 0.1 0 3 192 105 7 0.13533401489257812 0.09207510948181152 0.020456790924072266 0.010401010513305664 0.010055780410766602
f 0.1 0 4 192 99 7 0.22499680519104004 0.07315516471862793 0.014111757278442383 0.004890918731689453 0.00922083854675293
f 0.1 0 5 192 83 7 0.13685202598571777 0.06069183349609375 0.012955904006958008 0.0058629512786865234 0.007092952728271484
f 0.1 0 6 192 81 7 0.13498282432556152 0.05832695960998535 0.01238107681274414 0.005295991897583008 0.007085084915161133
f 0.1 0 7 192 75 7 0.1362311840057373 0.05388808250427246 0.012392997741699219 0.005350828170776367 0.0070421695709228516
f 0.1 0 8 192 70 7 0.136

In [17]:
obj = 'f'
setting = 'n100'
alpha = 0.1
trial = 0
t = 0

with open(f'data/{obj}-{setting}/o{alpha}/{trial*100+t}.pkl', 'rb') as f:
    G = pickle.load(f)

# Cold
itr_c, x = greedy(yinit, G)
print(x)

qsol, time_qc = solve_continuous_quadratic(G, yinit)

print(qsol)

[ 11.000  11.000  17.000  16.000  39.000  30.000  29.000  29.000  29.000
  25.000  22.000  4.000  35.000  35.000  8.000  34.000  41.000  41.000
  41.000  32.000  31.000  31.000  31.000  31.000  31.000  2.000  28.000
  19.000  27.000  27.000  27.000  27.000  32.000  32.000  32.000  33.000
  25.000  24.000  25.000  6.000  24.000  24.000  7.000  19.000  24.000
  24.000  24.000  24.000  24.000  24.000  24.000  24.000  8.000  13.000
  24.000  24.000  18.000  24.000  27.000  27.000  27.000  27.000  27.000
  24.000  31.000  31.000  31.000  46.000  26.000  50.000  33.000  19.000
  1.000  41.000  6.000  41.000  21.000  41.000  23.000  44.000  44.000
  44.000  24.000  44.000  30.000  29.000  29.000  4.000  9.000  38.000
  13.000  38.000  39.000  38.000  38.000  38.000  41.000  41.000  42.000
  14.000]
[ 10.400  11.600  16.500  16.500  39.000  29.281  29.281  29.281  29.156
  25.000  22.000  4.000  34.667  34.667  8.000  34.666  41.000  41.000
  41.000  31.250  31.250  31.250  31.250  31.000  31.